In [1]:
from IPython.display import display, Image, clear_output, HTML
import time
import random
import ipywidgets as widgets
import pandas as pd
import zipfile
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json 

In [2]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return


def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event(timeout=-1)
    text_input.disabled = True
    return event['description']


In [3]:
def run_ANS(results_dict):
    
    #Set and upload ANS images
    ANS0 = Image("12_9.png", width =500) #L
    ANS1 = Image("16_12.png", width =500) #L
    ANS2 = Image("20_15.png", width =500) #L
    ANS3 = Image("14_12.png", width =500) #L
    ANS4 = Image("21_18.png", width =500) #L
    ANS5 = Image("18_16.png", width =500) #L
    ANS6= Image("10_9.png", width =500) #L
    ANS7 = Image("20_18.png", width =500) #L

    ANS8 = Image("9_12.png", width =500) #R
    ANS9 = Image("12_16.png", width =500) #R
    ANS10 = Image("15_20.png", width =500) #R
    ANS11 = Image("12_14.png", width =500) #R
    ANS12 = Image("18_21.png", width =500) #R
    ANS13 = Image("16_18.png", width =500) #Rz
    ANS14 = Image("9_10.png", width =500) #R
    ANS15 = Image("18_20.png", width =500) #R

    #List ANS, filename, and ratio
    ANS_list = [ANS0, ANS1, ANS2, ANS3, ANS4, ANS5, ANS6, ANS7, ANS8, ANS9, ANS10, ANS11, ANS12, ANS13, ANS14, ANS15, ANS15]
    ANS_left =[ANS0, ANS2, ANS3, ANS4, ANS5, ANS6, ANS7, ANS7]
    ANS_right=[ANS8, ANS9, ANS10, ANS11, ANS12, ANS13, ANS14, ANS15]

    filename =[]
    for i in range (16):
        filename.append(ANS_list[i].filename)
        
    ratio= ['4:3', '4:3', '4:3','7:6', '7:6', '9:8', '10:9', '10:9', '3:4','3:4', '3:4', '6:7', '6:7', '9:8', '9:10', '9:10' ]

     #Zip ANS, filename, and ratio
    zip_result=zip(ANS_list, filename, ratio)
    zip_list=list(zip_result)
    

    #Make data dictionary
   

    # Set up each button (Left/Right)
    # to call the register_event
    # function when clicked
    btn1 = widgets.Button(description="Left")
    btn2 = widgets.Button(description="Right")

    btn1.on_click(register_event) 
    btn2.on_click(register_event) 
    
    # Reproducible random numbers
    random.seed(64)
    # Set an initial score 0
    score=0
    #Make correctnes list
    correctness=[]
    
    #Instruction 
    myhtml0 = HTML("<h1>Look at the following images and select which oval(left/right) has more dots by selecting the left or right button.</h1>")
    display(myhtml0)
    myhtml00 = HTML("<h1>You have 3 seconds to answer each question.</h1>")
    display(myhtml00)
    time.sleep(5)
    clear_output()

    # 4*16 Loops
    for n in range (4):
        for i in range(16):
            random.shuffle(zip_list)
            
            # Set dataframe
            mydataframe=pd.DataFrame (zip_list, columns = ['ANS','file name', 'ratio'])
        
            Extracted_ANS= mydataframe['ANS'].tolist()
            Extracted_filename= mydataframe['file name'].tolist()
            Extracted_ratio= mydataframe['ratio'].tolist()
            
            # Display dot image
            display(Extracted_ANS[i])
        
            # 0.75s per image
            time.sleep(0.75)
            clear_output()
            
            # start time
            start_time = time.time()
            
            # Question and answer
            myhtml1 = HTML("<h1>Which side had more number of dots?</h1>")
            display(myhtml1)
            myhtml2 = HTML("<h2>You have 3 seconds to answer</h2>")
            display(myhtml2)
                   
            panel = widgets.HBox([btn1, btn2])
            display(panel)
            result = wait_for_event(timeout=3)
            clear_output()
            
            # Scoring
            if result['description'] == "Left" and Extracted_ANS[i] in ANS_left:
                
                score += 1  
                correctness.append("1")
            elif result['description'] == "Right" and Extracted_ANS[i] in ANS_right:
                
                score += 1
                correctness.append("1")
            elif time.time() - start_time >= 3:  # 3s response time
                print("Time's up! You didn't answer.")
                score += 0
                correctness.append("0")
            else:
                
                score += 0
                correctness.append("0")
            
            # end time, time taken
            end_time = time.time()
            time_taken = end_time - start_time
            print(f'time taken: {time_taken:.2f} seconds')
            
            #Pause between images
            time.sleep(1.5)
            clear_output()
            
            rounded_time_taken = round(time_taken, 2)
        
            #Add to dictionary 
            results_dict['filename'].append(Extracted_filename[i])
            results_dict['ratio'].append(Extracted_ratio[i])
            results_dict['correct'].append(correctness[i])
            results_dict['response_time'].append(rounded_time_taken)
        


   

    #Print total score
    print(f'Your total score is: {score} /64 ({round(100*(score/64), 2)}%)')
    
    return score



In [4]:
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok





In [5]:
def run_full_test():
  

    results_dict = {
        'filename': [],
        'ratio': [],
        'correct': [],
        'response_time':[]
    }
    # Collecting variable data
   
    time.sleep(1)
    clear_output(wait=False)

    
    print("Please enter your age:")
    age = input()
    while len(str(age)) == 0:
        print("Invalid input! Age cannot be empty.")
        age = input("Please enter your age again: ")
    if int(age) < 0:
        print("Invalid input! Age cannot be negative.")
        age = input("Please enter your age again: ")

    time.sleep(1)
    clear_output(wait=False)
    
    print("What is your ethnicity?")
    
    # Create buttons with options
    btna1 = widgets.Button(description='White')
    btna2 = widgets.Button(description='Black or African American/British')
    btna3 = widgets.Button(description='Asian')
    btna4 = widgets.Button(description='Other')
    btna5 = widgets.Button(description='Rather not say')
    
    # Assign event handlers to buttons
    btna1.on_click(register_event) 
    btna2.on_click(register_event) 
    btna3.on_click(register_event) 
    btna4.on_click(register_event) 
    btna5.on_click(register_event) 

    display(btna1, btna2, btna3, btna4, btna5)
    event_infoa = wait_for_event(timeout=60)

    ethnicity = event_infoa['description']
    print(f"User clicked: {event_infoa['description']}")

    time.sleep(1)
    clear_output(wait=False)


    print("Please enter your biological sex:")
    
    btnb1 = widgets.Button(description='Male')
    btnb2 = widgets.Button(description='Female')
    btnb3 = widgets.Button(description='Rather not say')
    
    # Assign event handlers to buttons
    btnb1.on_click(register_event) 
    btnb2.on_click(register_event) 
    btnb3.on_click(register_event) 

    display(btnb1, btnb2, btnb3)
    event_infob = wait_for_event(timeout=60)

    sex = event_infob['description']
    print(f"User clicked: {event_infob['description']}")

    time.sleep(1)
    clear_output(wait=False)

    
    print("Did you have breakfast today:")
    btnc1 = widgets.Button(description='Yes')
    btnc2 = widgets.Button(description='No')
    btnc3 = widgets.Button(description='Rather not say')
    
    
    btnc1.on_click(register_event) 
    btnc2.on_click(register_event) 
    btnc3.on_click(register_event) 

    display(btnc1, btnc2, btnc3)
    event_infoc = wait_for_event(timeout=60)

    breakfast = event_infoc['description']
    print(f"User clicked: {event_infoc['description']}")

    time.sleep(1)
    clear_output(wait=False)
    
    print("How long did you sleep last night?")
    btnd1 = widgets.Button(description='less than 5 hours')
    btnd2 = widgets.Button(description='5-9 hours')
    btnd3 = widgets.Button(description='more than 9 hours')
    btnd4 = widgets.Button(description='Rather not say')
    
    # Assign event handlers to buttons
    btnd1.on_click(register_event) 
    btnd2.on_click(register_event) 
    btnd3.on_click(register_event) 
    btnd4.on_click(register_event) 

    display(btnd1, btnd2, btnd3, btnd4)
    event_infod = wait_for_event(timeout=60)

    sleep = event_infod['description']
    print(f"User clicked: {event_infod['description']}")

    time.sleep(1)
    clear_output(wait=False)

    
    score= run_ANS(results_dict)

    
    #Data dictionary 
    results_df = pd.DataFrame(results_dict)
    
    results_json=results_df.to_json()

    data_dict = {
    'user_id': user_id,
    'age': age,
    'ethnicity': ethnicity,
    'sex': sex,
    'breakfast': breakfast,
    'sleep': sleep,
    'total_score': round(100*(score/64), 2),
    'results_json': results_df.to_json()
    }

    #Data saving instruction
    data_consent_info = """DATA CONSENT INFORMATION:

    Please read:

    We wish to record your response data
    to an anonymised public data repository. 
    Your data will be used for educational teaching purposes
    practising data analysis and visualisation.

    Please type   yes   in the box below if you consent to the upload."""

    print(data_consent_info)
    result = text_input("> ") 
    if result == "yes" or "YES":
        print("Thanks for your participation.")
        print("Please contact a.fedorec@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")

        #send_to_google_form(data_dict, form_url
        
        form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSdBT5f3_u9lA3zd0qHG3JqFsPT0RQSE1SbvVt_xQ0MkjcSwGA/viewform?usp=sf_link'
        send_to_google_form(data_dict, form_url)
        
    else:
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue test."))


    return 


In [6]:
id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifier would be CBTC
"""



In [7]:
print(id_instructions)
user_id = input("> ")

print("User entered id:", user_id)

run_full_test()


KeyboardInterrupt

